In [375]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [376]:
import seaborn as sns


In [377]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [378]:
!pwd

/home/apratim941208/ganalytics


In [379]:
os.path.basename('train.csv')

'train.csv'

In [380]:
def load_df(csv_path='train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [381]:
pd.__version__

'0.23.4'

In [382]:
train_check = load_df()

Loaded train.csv. Shape: (903653, 55)


In [383]:
train_check.shape

(903653, 55)

In [384]:
test_check = load_df('test.csv')

Loaded test.csv. Shape: (804684, 53)


In [1]:
test_check['fullVisitorId'].nunique()

NameError: name 'test_check' is not defined

In [386]:
test_check['fullVisitorId'].dtype

dtype('O')

In [387]:
'0000130646294093000' in list(test_check['fullVisitorId'])

True

In [388]:
test_check.shape

(804684, 53)

In [389]:
test_check['fullVisitorId'].nunique()

617242

In [390]:
# train_check.to_csv('ganalytics_train_flattened.csv')

In [391]:
# test_check.to_csv('ganalytics_test_flattened.csv',index=False)

In [392]:
test_df = test_check.copy()

In [393]:
test_df['fullVisitorId'].nunique()

617242

In [394]:
train_df = train_check.copy()

In [395]:
train_df['fullVisitorId'].nunique()

714167

In [397]:
len((set(train_df['fullVisitorId'])).intersection((set(test_df['fullVisitorId']))))

7679

In [399]:
train_df.isnull().sum(),test_df.isnull().sum() 

(channelGrouping                                           0
 date                                                      0
 fullVisitorId                                             0
 sessionId                                                 0
 socialEngagementType                                      0
 visitId                                                   0
 visitNumber                                               0
 visitStartTime                                            0
 device.browser                                            0
 device.browserSize                                        0
 device.browserVersion                                     0
 device.deviceCategory                                     0
 device.flashVersion                                       0
 device.isMobile                                           0
 device.language                                           0
 device.mobileDeviceBranding                               0
 device.mobileDeviceInfo

In [400]:
train_df.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue

In [401]:
train_df =train_df.drop('trafficSource.campaignCode',axis=1)

In [402]:
def remove_constant_columns(df):
    columns = [c for c in df.columns if df[c].nunique(dropna=False)==1]
    df = df.drop(columns, axis=1)
    return df

In [403]:
train_df['totals.transactionRevenue']  =train_df['totals.transactionRevenue'].fillna(0.0)

In [404]:
train_df = remove_constant_columns(train_df)

In [405]:
test_df = remove_constant_columns(test_df)

In [406]:
train_df['channelGrouping'].isnull().count()

903653

In [407]:
for c in train_df.select_dtypes('object').columns.values:
    print(c)

channelGrouping
fullVisitorId
sessionId
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
totals.bounces
totals.hits
totals.newVisits
totals.pageviews
totals.transactionRevenue
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.isTrueDirect
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source


In [408]:
def categorical_missing_percentage(df):
    value = df[df.select_dtypes('object').columns].isnull().sum().sort_values(ascending=False)
    
    value2 = (df[df.select_dtypes('object').columns].isnull().sum()/df.shape[0]).sort_values(ascending=False)
    adding = pd.concat([value, value2],axis=1,join='outer', keys=['Values','Percentage'])
    return adding

In [409]:
categorical_missing_percentage(train_df),'                                            ', categorical_missing_percentage(test_df) 

(                                              Values  Percentage
 trafficSource.adContent                       892707    0.987887
 trafficSource.adwordsClickInfo.adNetworkType  882193    0.976252
 trafficSource.adwordsClickInfo.slot           882193    0.976252
 trafficSource.adwordsClickInfo.page           882193    0.976252
 trafficSource.adwordsClickInfo.isVideoAd      882193    0.976252
 trafficSource.adwordsClickInfo.gclId          882092    0.976140
 trafficSource.isTrueDirect                    629648    0.696781
 trafficSource.referralPath                    572712    0.633774
 trafficSource.keyword                         502929    0.556551
 totals.bounces                                453023    0.501324
 totals.newVisits                              200593    0.221980
 totals.pageviews                                 100    0.000111
 geoNetwork.city                                    0    0.000000
 device.operatingSystem                             0    0.000000
 device.de

In [410]:
import datetime as dt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [411]:
cal = calendar()

In [412]:
def add_date_features(df):
    
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
    
    holidays = cal.holidays(start = df['date'].min(), end = df['date'].max())
    
    df["month"]   = df['date'].dt.month
    df["day"]     = df['date'].dt.day
    df["weekday"] = df['date'].dt.weekday
    df['holiday_var'] = df['date'].isin(holidays)
    return df 

In [413]:
train_df = add_date_features(train_df)

In [414]:
test_df = add_date_features(test_df)

In [415]:
train_df.head(2)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,month,day,weekday,holiday_var
0,Organic Search,2016-09-02,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,(not set),NaN,(not provided),organic,NaN,google,9,2,4,False
1,Organic Search,2016-09-02,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,(not set),NaN,(not provided),organic,NaN,google,9,2,4,False


In [416]:
removing_features = ['date','sessionId','visitId','visitStartTime','visitNumber']

In [417]:
train_df = train_df.drop(removing_features,axis=1)

In [418]:
test_df = test_df.drop(removing_features,axis=1)

In [419]:
train_df.select_dtypes('int').columns

Index(['month', 'day', 'weekday'], dtype='object')

In [420]:
train_df.select_dtypes('float').columns

Index([], dtype='object')

In [421]:
train_df[train_df.select_dtypes('int').columns].describe()

,month,day,weekday
count,903653.000000,903653.000000,903653.000000
mean,6.987671,15.697570,2.735183
std,3.484576,8.823674,1.929072
min,1.000000,1.000000,0.000000
25%,4.000000,8.000000,1.000000
50%,7.000000,16.000000,3.000000
75%,10.000000,23.000000,4.000000
max,12.000000,31.000000,6.000000


In [422]:
test_df['totals.hits'] = test_df['totals.hits'].astype('float')

In [423]:
train_df['totals.hits'] = train_df['totals.hits'].astype('float')

In [424]:
def numerical_scaling(df):
           
    df['totals.hits'] = (df["totals.hits"] - min(df["totals.hits"]))/(max(df["totals.hits"]) - min(df["totals.hits"])) 
    
    df['totals.pageviews'] = df['totals.pageviews'].astype('float')
    df['totals.pageviews'] = (df["totals.pageviews"] - min(df["totals.pageviews"]))/(max(df["totals.pageviews"]) - min(df["totals.pageviews"])) 
    return df

In [425]:
train_df['totals.hits'].dtype

dtype('float64')

In [426]:
train_df['totals.pageviews'].dtype

dtype('O')

In [427]:
train_df = numerical_scaling(train_df)

In [428]:
test_df = numerical_scaling(test_df)

In [429]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [430]:
def add_numerical(df):
    
    df['totals.pageviews'] = df['totals.pageviews'].fillna(0.0)
    
    df['totals.bounces'] = df['totals.bounces'].astype(float)
    df['totals.bounces'] = df['totals.bounces'].fillna(0.0)
    
    df['totals.newVisits'] = df['totals.newVisits'].astype(float)
    df['totals.newVisits'] = df['totals.newVisits'].fillna(0.0)
    
    return df

In [431]:
train_df['totals.pageviews'].dtype

dtype('float64')

In [432]:
train_df = add_numerical(train_df)

In [433]:
test_df = add_numerical(test_df)

In [434]:
(train_df.isnull().sum().sort_values(ascending=False))

trafficSource.adContent                         892707
trafficSource.adwordsClickInfo.slot             882193
trafficSource.adwordsClickInfo.adNetworkType    882193
trafficSource.adwordsClickInfo.page             882193
trafficSource.adwordsClickInfo.isVideoAd        882193
trafficSource.adwordsClickInfo.gclId            882092
trafficSource.isTrueDirect                      629648
trafficSource.referralPath                      572712
trafficSource.keyword                           502929
geoNetwork.metro                                     0
geoNetwork.country                                   0
geoNetwork.continent                                 0
geoNetwork.city                                      0
holiday_var                                          0
geoNetwork.region                                    0
device.operatingSystem                               0
device.isMobile                                      0
device.deviceCategory                                0
device.bro

In [435]:
train_df['trafficSource.adwordsClickInfo.page'].value_counts()
#this is numerical

1     21362
2        73
3        10
5         7
7         3
9         2
4         2
14        1
Name: trafficSource.adwordsClickInfo.page, dtype: int64

In [436]:
train_df.select_dtypes('object').columns

Index(['channelGrouping', 'fullVisitorId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.transactionRevenue',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [437]:
from sklearn import preprocessing

In [438]:
def bool_feature(df):
    lbl = preprocessing.LabelEncoder()
    df['holiday_var'] = lbl.fit_transform(df['holiday_var'])
    df['device.isMobile'] = lbl.fit_transform(df['device.isMobile'])
    
    return df

In [439]:
train_df = bool_feature(train_df)

In [440]:
test_df = bool_feature(test_df)

In [441]:
train_df.select_dtypes('object').columns

Index(['channelGrouping', 'fullVisitorId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.transactionRevenue',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [442]:
categorical_columns = ['channelGrouping', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source']

In [443]:
train_df[categorical_columns].shape, test_df[categorical_columns].shape

((903653, 22), (804684, 22))

In [444]:
train_df[categorical_columns].isnull().sum()

channelGrouping                                      0
device.browser                                       0
device.deviceCategory                                0
device.operatingSystem                               0
geoNetwork.city                                      0
geoNetwork.continent                                 0
geoNetwork.country                                   0
geoNetwork.metro                                     0
geoNetwork.networkDomain                             0
geoNetwork.region                                    0
geoNetwork.subContinent                              0
trafficSource.adContent                         892707
trafficSource.adwordsClickInfo.adNetworkType    882193
trafficSource.adwordsClickInfo.gclId            882092
trafficSource.adwordsClickInfo.isVideoAd        882193
trafficSource.adwordsClickInfo.slot             882193
trafficSource.campaign                               0
trafficSource.isTrueDirect                      629648
trafficSou

In [445]:
train_df[categorical_columns].nunique().sort_values(ascending=False)

geoNetwork.networkDomain                        28064
trafficSource.adwordsClickInfo.gclId            17774
trafficSource.keyword                            3659
trafficSource.referralPath                       1475
geoNetwork.city                                   649
trafficSource.source                              380
geoNetwork.region                                 376
geoNetwork.country                                222
geoNetwork.metro                                   94
device.browser                                     54
trafficSource.adContent                            44
geoNetwork.subContinent                            23
device.operatingSystem                             20
trafficSource.campaign                             10
channelGrouping                                     8
trafficSource.medium                                7
geoNetwork.continent                                6
device.deviceCategory                               3
trafficSource.adwordsClickIn

In [446]:
X_train = train_df.copy()

In [447]:
X_test = test_df.copy()

In [448]:
X_train.shape, X_test.shape

((903653, 34), (804684, 33))

In [449]:
X_label = X_train['totals.transactionRevenue']

In [450]:
X_label.shape

(903653,)

In [451]:
df_combi = pd.concat([X_train,X_test], axis=0)

/home/apratim941208/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [452]:
df_combi.replace('nan', np.nan)

,channelGrouping,day,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,weekday
0,Organic Search,2,Chrome,desktop,0,Windows,1131660440785968503,Izmir,Asia,Turkey,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
1,Organic Search,2,Firefox,desktop,0,Macintosh,377306020877927890,not available in demo dataset,Oceania,Australia,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
2,Organic Search,2,Chrome,desktop,0,Windows,3895546263509774583,Madrid,Europe,Spain,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
3,Organic Search,2,UC Browser,desktop,0,Linux,4763447161404445595,not available in demo dataset,Asia,Indonesia,...,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google,4
4,Organic Search,2,Chrome,mobile,1,Android,27294437909732085,not available in demo dataset,Europe,United Kingdom,...,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google,4
5,Organic Search,2,Chrome,desktop,0,Windows,2938943183656635653,not available in demo dataset,Europe,Italy,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
6,Organic Search,2,Chrome,desktop,0,Windows,1905672039242460897,not available in demo dataset,Asia,Pakistan,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
7,Organic Search,2,Chrome,desktop,0,Windows,537222803633850821,Brisbane,Oceania,Australia,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
8,Organic Search,2,Internet Explorer,desktop,0,Windows,4445454811831400414,not available in demo dataset,Europe,Austria,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4
9,Organic Search,2,Firefox,desktop,0,Windows,9499785259412240342,not available in demo dataset,Europe,Netherlands,...,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,4


In [453]:
df_combi[categorical_columns].isnull().sum()

channelGrouping                                       0
device.browser                                        0
device.deviceCategory                                 0
device.operatingSystem                                0
geoNetwork.city                                       0
geoNetwork.continent                                  0
geoNetwork.country                                    0
geoNetwork.metro                                      0
geoNetwork.networkDomain                              0
geoNetwork.region                                     0
geoNetwork.subContinent                               0
trafficSource.adContent                         1643600
trafficSource.adwordsClickInfo.adNetworkType    1633063
trafficSource.adwordsClickInfo.gclId            1632914
trafficSource.adwordsClickInfo.isVideoAd        1633063
trafficSource.adwordsClickInfo.slot             1633063
trafficSource.campaign                                0
trafficSource.isTrueDirect                      

In [454]:
df_combi[categorical_columns].nunique().sort_values(ascending=False)

trafficSource.adwordsClickInfo.gclId            59008
geoNetwork.networkDomain                        41982
trafficSource.keyword                            5392
trafficSource.referralPath                       3196
geoNetwork.city                                   956
trafficSource.source                              500
geoNetwork.region                                 483
geoNetwork.country                                228
device.browser                                    129
geoNetwork.metro                                  123
trafficSource.adContent                            76
trafficSource.campaign                             35
device.operatingSystem                             24
geoNetwork.subContinent                            23
channelGrouping                                     8
trafficSource.medium                                7
geoNetwork.continent                                6
trafficSource.adwordsClickInfo.adNetworkType        3
trafficSource.adwordsClickIn

In [455]:
df_combi.shape

(1708337, 34)

In [456]:
small_cols = ['channelGrouping','trafficSource.medium','geoNetwork.continent','trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.slot',                 
'device.deviceCategory','trafficSource.adwordsClickInfo.isVideoAd','trafficSource.isTrueDirect']

In [457]:
df_combi[small_cols].isnull().sum()

channelGrouping                                       0
trafficSource.medium                                  0
geoNetwork.continent                                  0
trafficSource.adwordsClickInfo.adNetworkType    1633063
trafficSource.adwordsClickInfo.slot             1633063
device.deviceCategory                                 0
trafficSource.adwordsClickInfo.isVideoAd        1633063
trafficSource.isTrueDirect                      1173819
dtype: int64

In [458]:
small_columns = pd.get_dummies(df_combi[small_cols])

In [459]:
small_columns.shape

(1708337, 32)

In [460]:
df_combi.shape

(1708337, 34)

In [461]:
df_combi = pd.concat([df_combi,small_columns],axis=1)

In [462]:
df_combi = df_combi.drop(small_cols,axis=1)

In [463]:
df_combi.shape

(1708337, 58)

In [464]:
categorical_columns

['channelGrouping',
 'device.browser',
 'device.deviceCategory',
 'device.operatingSystem',
 'geoNetwork.city',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source']

In [465]:
small_cols

['channelGrouping',
 'trafficSource.medium',
 'geoNetwork.continent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.slot',
 'device.deviceCategory',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.isTrueDirect']

In [466]:
large_cols = list(set(categorical_columns)-set(small_cols))

In [467]:
df_combi[large_cols].shape


(1708337, 14)

In [468]:
df_combi[large_cols].isnull().sum()


device.operatingSystem                        0
trafficSource.source                          0
geoNetwork.region                             0
trafficSource.keyword                    893961
geoNetwork.city                               0
trafficSource.campaign                        0
geoNetwork.subContinent                       0
geoNetwork.networkDomain                      0
device.browser                                0
geoNetwork.country                            0
trafficSource.adContent                 1643600
geoNetwork.metro                              0
trafficSource.adwordsClickInfo.gclId    1632914
trafficSource.referralPath              1142073
dtype: int64

In [469]:
large_cols

['device.operatingSystem',
 'trafficSource.source',
 'geoNetwork.region',
 'trafficSource.keyword',
 'geoNetwork.city',
 'trafficSource.campaign',
 'geoNetwork.subContinent',
 'geoNetwork.networkDomain',
 'device.browser',
 'geoNetwork.country',
 'trafficSource.adContent',
 'geoNetwork.metro',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.referralPath']

In [470]:
large_cols = list(set(large_cols)-set(['trafficSource.adwordsClickInfo.gclId']))

In [471]:
df_combi[large_cols].shape


(1708337, 13)

In [472]:
df_combi['browser'] = np.where(df_combi['device.browser'].isin(['Chrome', 'Safari', 'Firefox', 'Internet Explorer', 'Android Webview','Edge', 'Samsung Internet', 'Opera Mini']),df_combi['device.browser'],'None')

In [473]:
df_combi = df_combi.drop('device.browser',axis=1)

In [474]:
df_combi['device.operatingSystem'].value_counts().head(8).index

Index(['Windows', 'Macintosh', 'Android', 'iOS', 'Linux', 'Chrome OS',
       '(not set)', 'Windows Phone'],
      dtype='object')

In [475]:
df_combi['os'] = np.where(df_combi['device.operatingSystem'].isin(['Windows', 'Macintosh', 'Android', 'iOS', 'Linux']),df_combi['device.operatingSystem'],'None')

In [476]:
df_combi = df_combi.drop('device.operatingSystem',axis=1)

In [477]:
def onehot(df):
    br = pd.get_dummies(df['browser'])
    osystem = pd.get_dummies(df['os'])
    
    df = pd.concat([df,br],axis=1)
    df = pd.concat([df,osystem],axis=1)
    
    df = df.drop('browser',axis=1)
    df = df.drop('os',axis=1)
    
    return df

In [478]:
df_combi = onehot(df_combi)

In [479]:
df_combi.columns

Index(['day', 'device.isMobile', 'fullVisitorId', 'geoNetwork.city',
       'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain',
       'geoNetwork.region', 'geoNetwork.subContinent', 'holiday_var', 'month',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.referralPath',
       'trafficSource.source', 'weekday', 'channelGrouping_(Other)',
       'channelGrouping_Affiliates', 'channelGrouping_Direct',
       'channelGrouping_Display', 'channelGrouping_Organic Search',
       'channelGrouping_Paid Search', 'channelGrouping_Referral',
       'channelGrouping_Social', 'trafficSource.medium_(none)',
       'trafficSource.medium_(not set)', 'trafficSource.medium_affiliate',
       'trafficSource.medium_cpc', 'trafficSou

In [480]:
df_combi = df_combi.drop('trafficSource.adContent',axis=1)

In [481]:
large_cols = list(set(large_cols)-set(['device.operatingSystem','device.browser']))

In [482]:
large_cols = list(set(large_cols) - set(['trafficSource.adContent']))

In [483]:
large_cols

['trafficSource.source',
 'geoNetwork.region',
 'trafficSource.campaign',
 'geoNetwork.city',
 'geoNetwork.networkDomain',
 'geoNetwork.subContinent',
 'geoNetwork.country',
 'trafficSource.keyword',
 'geoNetwork.metro',
 'trafficSource.referralPath']

In [484]:
df_combi = df_combi.drop('trafficSource.referralPath',axis=1)

In [485]:
large_cols = list(set(large_cols) - set(['trafficSource.referralPath']))

In [486]:
df_combi[large_cols].isnull().sum()

trafficSource.source             0
geoNetwork.region                0
geoNetwork.city                  0
trafficSource.campaign           0
geoNetwork.networkDomain         0
geoNetwork.subContinent          0
geoNetwork.country               0
trafficSource.keyword       893961
geoNetwork.metro                 0
dtype: int64

In [487]:
df_combi['trafficSource.keyword'] = np.where(df_combi['trafficSource.keyword'] != '(not provided)',0,1)

In [488]:
large_cols = list(set(large_cols) - set(['trafficSource.keyword']))

In [489]:
large_cols

['trafficSource.source',
 'geoNetwork.region',
 'trafficSource.campaign',
 'geoNetwork.city',
 'geoNetwork.networkDomain',
 'geoNetwork.subContinent',
 'geoNetwork.country',
 'geoNetwork.metro']

In [490]:
df_combi[large_cols].nunique()

trafficSource.source          500
geoNetwork.region             483
trafficSource.campaign         35
geoNetwork.city               956
geoNetwork.networkDomain    41982
geoNetwork.subContinent        23
geoNetwork.country            228
geoNetwork.metro              123
dtype: int64

In [500]:
df_combi['totals.transactionRevenue'] = df_combi['totals.transactionRevenue'].fillna(0.0)

In [503]:
df_combi['totals.transactionRevenue'] = df_combi['totals.transactionRevenue'].astype(float)

In [504]:
df_combi['fullVisitorId'].dtypes

dtype('O')

In [505]:
df_combi.groupby(['geoNetwork.subContinent'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.subContinent'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.subContinent
Polynesia             0.000000
Middle Africa         0.000000
Micronesian Region    0.000000
Melanesia             0.000000
Central Asia          0.000023
Northern Africa       0.000033
Southern Africa       0.000043
Southern Asia         0.000490
(not set)             0.000500
Eastern Europe        0.000615
Southern Europe       0.000626
Caribbean             0.001015
Western Asia          0.001156
Australasia           0.001164
Central America       0.001237
Northern Europe       0.001295
Western Europe        0.001846
Western Africa        0.002144
Southeast Asia        0.002439
Eastern Africa        0.003421
Eastern Asia          0.007191
South America         0.010348
Northern America      0.964413
Name: totals.transactionRevenue, dtype: float64

In [506]:
df_combi['geoNetwork.subContinent'] = np.where(df_combi['geoNetwork.subContinent'] != 'Northern America',0,1)

In [507]:
df_combi.groupby(['geoNetwork.networkDomain'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.networkDomain'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.networkDomain
lego.com                   0.000000
parkalgar.com              0.000000
parkbilling.com            0.000000
parkchambers.co.nz         0.000000
parkcity.ne.jp             0.000000
parkdns.work               0.000000
parkhillgroup.com          0.000000
parking-page.net           0.000000
park2000.co.za             0.000000
parkinson-spencer.co.uk    0.000000
parkland.edu               0.000000
parklandsd.org             0.000000
parklink.com               0.000000
parknet.com                0.000000
parknet.dk                 0.000000
parknicollet.com           0.000000
parkpost.tv                0.000000
parkinsonalberta.ca        0.000000
parkroyalstudios.com       0.000000
park.se                    0.000000
park-web.net               0.000000
paraiso-peru.com           0.000000
paraisopolis.com.br        0.000000
paramatrix.co.in           0.000000
paramountgroup.co.za       0.000000
paranaweb.com.br           0.000000
parbo.net                  0.000000
par

In [508]:
df_combi['geoNetwork.networkDomain'] = np.where(df_combi['geoNetwork.networkDomain'] != '(not set)',0,1)

In [509]:
df_combi.groupby(['geoNetwork.city'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.city'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.city
's Hertogenbosch                 0.000000
Nuremberg                        0.000000
Ocala                            0.000000
Odense                           0.000000
Odessa                           0.000000
Oegstgeest                       0.000000
Okemos                           0.000000
Oklahoma City                    0.000000
Olomouc                          0.000000
Oneonta                          0.000000
Or Yehuda                        0.000000
Oradea                           0.000000
Orebro                           0.000000
Orem                             0.000000
Orihuela                         0.000000
Orleans                          0.000000
Osaka                            0.000000
Oshkosh                          0.000000
Osijek                           0.000000
Oslo                             0.000000
Oslo Municipality                0.000000
Ostrava                          0.000000
Ota                              0.000000
Ottignies-Louvain-

In [510]:
def city(fx):
    if fx=='not available in demo dataset':
        fx=0
    elif fx=='New York':
        fx=1
    else:
        fx=2
    return fx

In [511]:
df_combi['geoNetwork.city'] = df_combi['geoNetwork.city'].apply(city)

In [512]:
df_combi.groupby(['geoNetwork.region'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.region'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.region
Malacca                          0.000000
Pays de la Loire                 0.000000
Pathum Thani                     0.000000
Panama                           0.000000
Overijssel                       0.000000
Ouest Department                 0.000000
Oslo                             0.000000
Osijek-Baranja County            0.000000
Osaka Prefecture                 0.000000
Orebro County                    0.000000
Oran Province                    0.000000
West Virginia                    0.000000
Olomouc Region                   0.000000
Oklahoma                         0.000000
Okinawa Prefecture               0.000000
Odisha                           0.000000
Odessa Oblast                    0.000000
Occitanie                        0.000000
Nuevo Leon                       0.000000
Nouvelle-Aquitaine               0.000000
Northern Ireland                 0.000000
North Sumatra                    0.000000
North Rhine-Westphalia           0.000000
North Denmark Re

In [513]:
def region(fx):
    if fx=='not available in demo dataset':
        fx=0
    elif fx=='California':
        fx=1
    elif fx=='New York':
        fx=2
    else:
        fx=3
    return fx

In [514]:
df_combi['geoNetwork.region'] = df_combi['geoNetwork.region'].apply(region)

In [515]:
df_combi.groupby(['trafficSource.campaign'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['trafficSource.campaign'])['totals.transactionRevenue'].sum().sort_values().sum()

trafficSource.campaign
"google + redesign/Accessories March 17" 1 Day                          0.000000
Technology/Technophiles                                                 0.000000
Technology/Mobile Enthusiasts                                           0.000000
Sports & Fitness/Health & Fitness Buffs                                 0.000000
Smart Display Campaign                                                  0.000000
Run of Network Line Item                                                0.000000
Page: contains "/google+redesign/office" Similar Audiences              0.000000
Page: contains "/google+redesign/office"                                0.000000
Page: contains "/google+redesign/electronics" Similar Audiences         0.000000
Page: contains "/google+redesign/electronics"                           0.000000
Page: contains "/google+redesign/drinkware" Similar Audiences           0.000000
Page: contains "/google+redesign/drinkware"                             0.000000
Page:

In [516]:
def campaign(fx):
    if fx=='(not set)':
        fx=0
    else:
        fx=1
    return fx

In [517]:
df_combi['trafficSource.campaign'] = df_combi['trafficSource.campaign'].apply(campaign)

In [518]:
df_combi.groupby(['trafficSource.source'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['trafficSource.source'])['totals.transactionRevenue'].sum().sort_values().sum()

trafficSource.source
google.es                                 0.000000
learn.pct.edu                             0.000000
learn.colorado.edu                        0.000000
kristiyan.users.x20web.corp.google.com    0.000000
kr.pinterest.com                          0.000000
kik.com                                   0.000000
kidrex.org                                0.000000
ixquick.com                               0.000000
it.search.yahoo.com                       0.000000
it.pinterest.com                          0.000000
it-mg42.mail.yahoo.com                    0.000000
ipv6.google.com                           0.000000
ipv4.google.com                           0.000000
invincible.mtv.corp.google.com:8080       0.000000
intranet                                  0.000000
int.search.tb.ask.com                     0.000000
int.search.mywebsearch.com                0.000000
info.com                                  0.000000
inbox.google.com                          0.000000
in.search.

In [519]:
def source(fx):
    if fx=='mall.googleplex.com':
        fx=0
    elif fx=='(direct)':
        fx=1
    elif fx=='google':
        fx=2
    else:
        fx=3
    return fx

In [520]:
df_combi['trafficSource.source'] = df_combi['trafficSource.source'].apply(source)

In [521]:
df_combi.groupby(['geoNetwork.metro'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.metro'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.metro
Lexington KY                                               0.000000
North East                                                 0.000000
Norfolk-Portsmouth-Newport News VA                         0.000000
Wichita-Hutchinson KS                                      0.000000
New Orleans LA                                             0.000000
Monterey-Salinas CA                                        0.000000
Midlands                                                   0.000000
Meridian (exc. Channel Islands)                            0.000000
Memphis TN                                                 0.000000
Mankato MN                                                 0.000000
Madison WI                                                 0.000000
Lubbock TX                                                 0.000000
Louisville KY                                              0.000000
Yorkshire                                                  0.000000
Lincoln & Hastings-Kearney NE  

In [522]:
def metro(fx):
    if fx=='not available in demo dataset':
        fx=0
    elif fx=='San Francisco-Oakland-San Jose CA':
        fx=1
    elif fx=='New York NY':
        fx=2
    else:
        fx=3
    return fx

In [523]:
df_combi['geoNetwork.metro'] = df_combi['geoNetwork.metro'].apply(metro)

In [524]:
df_combi = df_combi.drop('geoNetwork.metro',axis=1)

In [525]:
df_combi.groupby(['geoNetwork.country'])['totals.transactionRevenue'].sum().sort_values()/df_combi.groupby(['geoNetwork.country'])['totals.transactionRevenue'].sum().sort_values().sum()

geoNetwork.country
Lesotho                     0.000000
Mauritania                  0.000000
Mauritius                   0.000000
Mayotte                     0.000000
Micronesia                  0.000000
Moldova                     0.000000
Monaco                      0.000000
Mongolia                    0.000000
Montenegro                  0.000000
Montserrat                  0.000000
Morocco                     0.000000
Mozambique                  0.000000
Myanmar (Burma)             0.000000
Namibia                     0.000000
Nepal                       0.000000
New Caledonia               0.000000
Niger                       0.000000
Norfolk Island              0.000000
Northern Mariana Islands    0.000000
Norway                      0.000000
Martinique                  0.000000
Oman                        0.000000
Marshall Islands            0.000000
Mali                        0.000000
Isle of Man                 0.000000
Jamaica                     0.000000
Jersey             

In [526]:
def country(fx):
    if fx=='United States':
        fx=0
    else:
        fx=1
    return fx

In [527]:
df_combi['geoNetwork.country'] = df_combi['geoNetwork.country'].apply(country)

/home/apratim941208/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [528]:
df_combi.dtypes

day                                                               int64
device.isMobile                                                   int64
fullVisitorId                                                    object
geoNetwork.city                                                   int64
geoNetwork.country                                                int64
geoNetwork.networkDomain                                          int64
geoNetwork.region                                                 int64
geoNetwork.subContinent                                           int64
holiday_var                                                       int64
month                                                             int64
totals.bounces                                                  float64
totals.hits                                                     float64
totals.newVisits                                                float64
totals.pageviews                                                

In [529]:
df_combi= df_combi.drop(['trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.page'],axis=1)

In [530]:
# df_combi['totals.bounces'] = df_combi['totals.bounces'].astype('float')

In [531]:
# df_combi['totals.hits'] = df_combi['totals.hits'].astype('float')

In [532]:
train=df_combi[:len(train_df)]
test=df_combi[len(train_df):]

In [533]:
train.shape, test.shape

((903653, 66), (804684, 66))

In [534]:
train.shape

(903653, 66)

In [535]:
train.shape, test.shape

((903653, 66), (804684, 66))

In [536]:
train.isnull().any().sum(), test.isnull().any().sum()

(0, 0)

In [539]:

col = list(train.columns)
col[-3] = 'None2'
train.columns = col

In [540]:
col2 = list(test.columns)
col2[-3] = 'None2'
test.columns = col2

In [543]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import lightgbm as lgb 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import model_selection, preprocessing, metrics

In [544]:
train.columns

Index(['day', 'device.isMobile', 'fullVisitorId', 'geoNetwork.city',
       'geoNetwork.country', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'holiday_var', 'month', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.source', 'weekday',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'trafficSource.medium_(none)', 'trafficSource.medium_(not set)',
       'trafficSource.medium_affiliate', 'trafficSource.medium_cpc',
       'trafficSource.medium_cpm', 'trafficSource.medium_organic',
       'trafficSource.medium_referral', 'geoNetwork.continent_(not set)',
       'geoNetwork.continent_Africa', 'geoNetwo

In [545]:
train['totals.transactionRevenue'] = np.log1p(train['totals.transactionRevenue'])

/home/apratim941208/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [546]:
# train['fullVisitorId'] = train['fullVisitorId'].astype('float')

In [547]:
# test['fullVisitorId'] = test['fullVisitorId'].astype('float')

In [548]:
train_vis_id = train['fullVisitorId']

In [549]:
test_vis_id = test['fullVisitorId']

In [550]:
test_vis_id.dtypes, train_vis_id.dtypes

(dtype('O'), dtype('O'))

In [551]:
train = train.drop('fullVisitorId', axis=1)

In [552]:
test = test.drop('fullVisitorId',axis=1)

In [555]:
train.columns

Index(['day', 'device.isMobile', 'geoNetwork.city', 'geoNetwork.country',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'holiday_var', 'month', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.source', 'weekday',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'trafficSource.medium_(none)', 'trafficSource.medium_(not set)',
       'trafficSource.medium_affiliate', 'trafficSource.medium_cpc',
       'trafficSource.medium_cpm', 'trafficSource.medium_organic',
       'trafficSource.medium_referral', 'geoNetwork.continent_(not set)',
       'geoNetwork.continent_Africa', 'geoNetwork.continent_Amer

In [556]:
train.head(4)

,day,device.isMobile,geoNetwork.city,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,holiday_var,month,totals.bounces,...,None,Opera Mini,Safari,Samsung Internet,Android,Linux,Macintosh,None2,Windows,iOS
0,2,0,2,1,0,3,0,0,9,1.0,...,0,0,0,0,0,0,0,0,1,0
1,2,0,0,1,0,0,0,0,9,1.0,...,0,0,0,0,0,0,1,0,0,0
2,2,0,2,1,0,3,0,0,9,1.0,...,0,0,0,0,0,0,0,0,1,0
3,2,0,0,1,0,0,0,0,9,1.0,...,1,0,0,0,0,1,0,0,0,0


In [557]:
features = list(['day', 'device.isMobile', 'geoNetwork.city', 'geoNetwork.country',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'holiday_var', 'month', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.source', 'weekday',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'trafficSource.medium_(none)', 'trafficSource.medium_(not set)',
       'trafficSource.medium_affiliate', 'trafficSource.medium_cpc',
       'trafficSource.medium_cpm', 'trafficSource.medium_organic',
       'trafficSource.medium_referral', 'geoNetwork.continent_(not set)',
       'geoNetwork.continent_Africa', 'geoNetwork.continent_Americas',
       'geoNetwork.continent_Asia', 'geoNetwork.continent_Europe',
       'geoNetwork.continent_Oceania',
       'trafficSource.adwordsClickInfo.adNetworkType_Content',
       'trafficSource.adwordsClickInfo.adNetworkType_Google Search',
       'trafficSource.adwordsClickInfo.adNetworkType_Search partners',
       'trafficSource.adwordsClickInfo.slot_Google Display Network',
       'trafficSource.adwordsClickInfo.slot_RHS',
       'trafficSource.adwordsClickInfo.slot_Top',
       'device.deviceCategory_desktop', 'device.deviceCategory_mobile',
       'device.deviceCategory_tablet',
       'trafficSource.adwordsClickInfo.isVideoAd_False',
       'trafficSource.isTrueDirect_True', 'Android Webview', 'Chrome', 'Edge',
       'Firefox', 'Internet Explorer', 'None', 'Opera Mini', 'Safari',
       'Samsung Internet', 'Android', 'Linux', 'Macintosh', 'None2', 'Windows',
       'iOS'])

In [558]:
train.shape, train[features].shape

((903653, 65), (903653, 64))

In [559]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals.transactionRevenue"], test_size=0.25, random_state=20)

In [560]:
lgb_params = {"objective" : "regression", "metric" : "rmse",
             "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
lgb_train = lgb.Dataset(train_x, label=train_y)
lgb_val = lgb.Dataset(valid_x, label=valid_y)
model = lgb.train(lgb_params, lgb_train, 1200, valid_sets=[lgb_val], early_stopping_rounds=150, verbose_eval=20)

Training until validation scores don't improve for 150 rounds.
[20]	valid_0's rmse: 1.84786
[40]	valid_0's rmse: 1.75846
[60]	valid_0's rmse: 1.70923
[80]	valid_0's rmse: 1.68043
[100]	valid_0's rmse: 1.66427
[120]	valid_0's rmse: 1.65361
[140]	valid_0's rmse: 1.64677
[160]	valid_0's rmse: 1.642
[180]	valid_0's rmse: 1.63797
[200]	valid_0's rmse: 1.63525
[220]	valid_0's rmse: 1.6332
[240]	valid_0's rmse: 1.6318
[260]	valid_0's rmse: 1.63053
[280]	valid_0's rmse: 1.62966
[300]	valid_0's rmse: 1.6289
[320]	valid_0's rmse: 1.62818
[340]	valid_0's rmse: 1.62739
[360]	valid_0's rmse: 1.62695
[380]	valid_0's rmse: 1.62651
[400]	valid_0's rmse: 1.62597
[420]	valid_0's rmse: 1.62571
[440]	valid_0's rmse: 1.62557
[460]	valid_0's rmse: 1.62537
[480]	valid_0's rmse: 1.62528
[500]	valid_0's rmse: 1.6251
[520]	valid_0's rmse: 1.62515
[540]	valid_0's rmse: 1.62506
[560]	valid_0's rmse: 1.62492
[580]	valid_0's rmse: 1.6247
[600]	valid_0's rmse: 1.62467
[620]	valid_0's rmse: 1.62468
[640]	valid_0's rm

In [561]:
import pickle

In [562]:
test_vis_id.dtype, test_vis_id.shape

(dtype('O'), (804684,))

In [563]:
test.shape

(804684, 65)

,day,device.isMobile,geoNetwork.city,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,holiday_var,month,totals.bounces,...,None,Opera Mini,Safari,Samsung Internet,Android,Linux,Macintosh,None2,Windows,iOS
0,16,0,2,1,0,3,0,0,10,0.0,...,0,0,0,0,0,0,1,0,0,0
1,16,0,2,1,0,3,0,0,10,0.0,...,0,0,0,0,0,0,0,0,1,0


In [565]:
preds = model.predict(test[features], num_iteration=model.best_iteration)
test["PredictedLogRevenue"] = np.expm1(preds)


In [566]:
test['fullVisitorId'] = test_vis_id

In [567]:
test.dtypes

day                                                               int64
device.isMobile                                                   int64
geoNetwork.city                                                   int64
geoNetwork.country                                                int64
geoNetwork.networkDomain                                          int64
geoNetwork.region                                                 int64
geoNetwork.subContinent                                           int64
holiday_var                                                       int64
month                                                             int64
totals.bounces                                                  float64
totals.hits                                                     float64
totals.newVisits                                                float64
totals.pageviews                                                float64
totals.transactionRevenue                                       

In [568]:
submission = test.groupby("fullVisitorId").agg({"PredictedLogRevenue" : "sum"}).reset_index()
submission["PredictedLogRevenue"] = np.log1p(submission["PredictedLogRevenue"])
submission["PredictedLogRevenue"] =  submission["PredictedLogRevenue"].apply(lambda x : 0.0 if x < 0 else x)


/home/apratim941208/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


In [569]:
submission.dtypes

fullVisitorId           object
PredictedLogRevenue    float64
dtype: object

In [570]:
submission.shape

(617242, 2)

In [571]:
submission.isnull().sum()

fullVisitorId          0
PredictedLogRevenue    8
dtype: int64

In [572]:
submission['PredictedLogRevenue'] = submission['PredictedLogRevenue'].fillna(0.0)

In [573]:
submission.to_csv("baseline_lgb.csv", index=False)
submission.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.316812
1,0000049363351866189,0.000000
2,0000053049821714864,0.000000
3,0000059488412965267,0.000000
4,0000085840370633780,0.000000


In [336]:
test_vis_id.unique().shape

(617242,)

In [342]:
'0000130646294093000' in list(test_vis_id)

True

In [311]:
test_vis_id.head(5)

0    6167871330617112363
1    0643697640977915618
2    6059383810968229466
3    2376720078563423631
4    2314544520795440038
Name: fullVisitorId, dtype: object

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.188578
1,0000049363351866189,0.000000
2,0000053049821714864,0.000000
3,0000059488412965267,0.000000
4,0000085840370633780,0.000000


In [347]:
train.shape, test.shape

((903653, 66), (804684, 66))

In [361]:
submission.shape

(617242, 2)

In [444]:
test_df['fullVisitorId'].dtype, test['fullVisitorId'].dtype

(dtype('O'), dtype('float64'))

In [445]:
len(set(test_df['fullVisitorId']).intersection(set(test['fullVisitorId'])))

0

In [367]:
submission.to_csv("baseline_lgb.csv", index=False)

In [370]:
submission_new = pd.DataFrame(submission)

In [446]:
submission_new.tail(10)

,fullVisitorId,PredictedLogRevenue
617232,8.398477224797657e+19,0.003694
617233,8.473852272867009e+19,0.003765
617234,8.875697195081336e+19,0.003765
617235,9.049160394281915e+19,0.003775
617236,9.300581812882661e+19,0.003758
617237,9.369797024712591e+19,0.001638
617238,9.406859810512901e+19,0.003694
617239,9.445575813421752e+19,0.000783
617240,9.490734873447455e+19,0.003765
617241,9.891190212878238e+19,0.003765


In [447]:
submission_new.isnull().sum()

fullVisitorId          0
PredictedLogRevenue    0
dtype: int64

In [386]:
submission_new['PredictedLogRevenue'] = submission_new['PredictedLogRevenue'].fillna(0.0)

In [461]:
submission_new.nunique()

fullVisitorId          617242
PredictedLogRevenue     85528
dtype: int64

In [462]:
submission_new.isnull().sum()

fullVisitorId          0
PredictedLogRevenue    0
dtype: int64

In [463]:
submission_new.to_csv('baseline_lgb.csv',index=False)

In [465]:
!pip install --user kaggle

In [464]:
!kaggle competitions submit -c ga-customer-revenue-prediction -f baseline_lgb.csv -m "LGB"

401 - Unauthorized


In [393]:
test['fullVisitorId'].any()<0

False

In [413]:
['0000000259678714014'] in (list(submission_new['fullVisitorId']))

False

In [414]:
['0000000259678714014'] in (list(test['fullVisitorId']))

False

In [416]:
['0000000259678714014'] in (list(test_check['fullVisitorId']))

False

In [420]:
submission_new['fullVisitorId'] = submission_new['fullVisitorId'].astype(str)

In [424]:
submission_new.dtypes

fullVisitorId           object
PredictedLogRevenue    float64
dtype: object

In [429]:
submission_new.isna().any()

fullVisitorId          False
PredictedLogRevenue    False
dtype: bool

In [380]:
submission.isnull().sum()

fullVisitorId           0
PredictedLogRevenue    17
dtype: int64

In [331]:
submission['PredictedLogRevenue'] = submission['PredictedLogRevenue'].fillna(0.0)

In [337]:
submission['fullVisitorId'] = submission['fullVisitorId'].astype('int')

In [442]:
submission_new.to_csv("baseline_lgb.csv", index=False)
submission.head()

,fullVisitorId,PredictedLogRevenue
0,2.596787e+11,0.199390
1,3.038793e+12,0.000000
2,9.663019e+12,0.000000
3,1.043846e+13,0.010134
4,1.099298e+13,0.000233


In [431]:
submission_new['fullVisitorId'].nunique()

617242

In [453]:
['53049821714864'] in list(test['fullVisitorId'])

False

In [454]:
['0000000259678714014'] in list(test_check['fullVisitorId'])

False

In [438]:
submission_new.nunique()

fullVisitorId          617242
PredictedLogRevenue     85528
dtype: int64

In [437]:
submission_new.isnull().sum()

fullVisitorId          0
PredictedLogRevenue    0
dtype: int64

In [443]:
submission_new.dtypes

fullVisitorId           object
PredictedLogRevenue    float64
dtype: object

In [ ]:
submission

In [310]:
filename= 'lgbmodel1.sav'

In [311]:
pickle.dump(model, open(filename, 'wb'))

In [323]:
test.shape

(804684, 65)

In [312]:
import xgboost as xgb

In [313]:
features = ['day', 'device.isMobile', 'fullVisitorId', 'geoNetwork.city',
       'geoNetwork.country', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'holiday_var', 'month', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.source', 'weekday',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'trafficSource.medium_(none)', 'trafficSource.medium_(not set)',
       'trafficSource.medium_affiliate', 'trafficSource.medium_cpc',
       'trafficSource.medium_cpm', 'trafficSource.medium_organic',
       'trafficSource.medium_referral', 'geoNetwork.continent_(not set)',
       'geoNetwork.continent_Africa', 'geoNetwork.continent_Americas',
       'geoNetwork.continent_Asia', 'geoNetwork.continent_Europe',
       'geoNetwork.continent_Oceania',
       'trafficSource.adwordsClickInfo.adNetworkType_Content',
       'trafficSource.adwordsClickInfo.adNetworkType_Google Search',
       'trafficSource.adwordsClickInfo.adNetworkType_Search partners',
       'trafficSource.adwordsClickInfo.slot_Google Display Network',
       'trafficSource.adwordsClickInfo.slot_RHS',
       'trafficSource.adwordsClickInfo.slot_Top',
       'device.deviceCategory_desktop', 'device.deviceCategory_mobile',
       'device.deviceCategory_tablet',
       'trafficSource.adwordsClickInfo.isVideoAd_False',
       'trafficSource.isTrueDirect_True', 'Android Webview', 'Chrome', 'Edge',
       'Firefox', 'Internet Explorer', 'None', 'Opera Mini', 'Safari',
       'Samsung Internet', 'Android', 'Linux', 'Macintosh', 'None2', 'Windows',
       'iOS']

In [314]:
data_dmatrix = xgb.DMatrix(data=train[features], label=train['totals.transactionRevenue'])

In [315]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals.transactionRevenue"], test_size=0.25, random_state=20)

In [316]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.8, learning_rate = 0.02,
                max_depth = 50, n_estimators = 50)

In [317]:
xg_reg.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=50, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [318]:
preds = xg_reg.predict(valid_x)

In [319]:
rmse = np.sqrt(mean_squared_error(valid_y, preds))
print("RMSE: %f" % (rmse))

RMSE: 1.745692


In [320]:
filename2 = 'xgb1.sav'

In [321]:
pickle.dump(xg_reg, open(filename2, 'wb'))

In [154]:
data_dmatrix

In [160]:
params = {"objective":"reg:linear",'colsample_bytree': 0.8,'learning_rate': 0.02,
                'max_depth': 40}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=25,early_stopping_rounds=8,metrics="rmse", as_pandas=True, seed=123)

[20:36:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19444 extra nodes, 0 pruned nodes, max_depth=31
[20:36:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19424 extra nodes, 0 pruned nodes, max_depth=36
[20:37:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19278 extra nodes, 0 pruned nodes, max_depth=33
[20:37:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19632 extra nodes, 0 pruned nodes, max_depth=31
[20:37:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19430 extra nodes, 0 pruned nodes, max_depth=32
[20:37:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17376 extra nodes, 0 pruned nodes, max_depth=31
[20:37:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18306 extra nodes, 0 pruned nodes, max_depth=33
[20:37:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17682 extra nodes, 0 pruned node

[20:45:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16940 extra nodes, 0 pruned nodes, max_depth=34
[20:45:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 19152 extra nodes, 0 pruned nodes, max_depth=31
[20:45:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18900 extra nodes, 0 pruned nodes, max_depth=40
[20:45:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17120 extra nodes, 0 pruned nodes, max_depth=32
[20:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17282 extra nodes, 0 pruned nodes, max_depth=33
[20:45:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17962 extra nodes, 0 pruned nodes, max_depth=39
[20:45:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17600 extra nodes, 0 pruned nodes, max_depth=30
[20:46:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16938 extra nodes, 0 pruned node

In [161]:
cv_results.tail(1)

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
24,1.408157,0.009779,1.826676,0.010868


In [163]:
pd.DataFrame(cv_results).to_csv('xgb_cv.csv', encoding='utf-8')